In [1]:
%pip install bettercam
%pip install dxcam 
%pip install pyvjoystick
%pip install torch torchvision opencv-python numpy pandas matplotlib albumentations
%pip install ultralytics  # (for YOLO)
%pip install keyboard
%pip install pyaccsharedmemory
%pip install mss

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


screen capture 

In [1]:
import dxcam
import os
import time
import datetime
import keyboard
from PIL import Image
from pyaccsharedmemory import accSharedMemory

# Output settings
OUTPUT_DIR = "ac_screenshots"
os.makedirs(OUTPUT_DIR, exist_ok=True)
SCREEN_REGION = (0, 0, 1920, 1080)

class ScreenshotCapture:
    def __init__(self):
        """Initialize screenshot capture and telemetry reader"""
        self.camera = dxcam.create()
        if self.camera is None:
            raise RuntimeError("Failed to create DXCam camera")
        
        # Initialize Assetto Corsa shared memory
        self.asm = accSharedMemory()
        print("DXCam and AC telemetry initialized successfully")

    def read_telemetry_data(self):
        """Read telemetry data from Assetto Corsa shared memory"""
        try:
            sm = self.asm.read_shared_memory()
            
            if sm is not None:
                return {
                    'speed': sm.Physics.speed_kmh,
                    'steering': sm.Physics.steer_angle,
                    'throttle': sm.Physics.gas,
                    'brake': sm.Physics.brake,
                    # 'rpm': sm.Physics.rpm,
                    # 'gear': sm.Physics.gear,
                    # 'fuel': getattr(sm.Physics, 'fuel', 0.0),
                    # 'lap_time': getattr(sm.Graphics, 'current_time', 0),
                    # 'position': getattr(sm.Graphics, 'position', 0)
                }
            return None
        except Exception as e:
            print(f"Error reading telemetry: {e}")
            return None

    def capture_screenshot(self):
        """Capture screenshot with telemetry data"""
        telemetry = self.read_telemetry_data()
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-3]
        
        # Handle telemetry data
        if telemetry:
            speed = telemetry['speed']
            brake = telemetry['brake']
            steering = telemetry['steering']
            throttle = telemetry['throttle']
            # gear = telemetry['gear']
            # rpm = telemetry['rpm']
            
            filename = (f"{OUTPUT_DIR}/ac_{timestamp}_"
                       f"speed_{speed:.1f}_brake_{brake:.2f}_"
                       f"steer_{steering:.3f}_throttle_{throttle:.2f}.png")
        else:
            # Fallback when no telemetry available
            filename = f"{OUTPUT_DIR}/ac_{timestamp}_no_telemetry.png"
        
        # Capture screenshot
        frame = self.camera.grab(region=SCREEN_REGION)
        if frame is not None:
            img = Image.fromarray(frame)
            img.save(filename)
            print(f"Saved {filename}")
            
            # Display telemetry info
            if telemetry:
                print(f"  Speed: {telemetry['speed']:.1f} km/h")
                print(f"  Steering: {telemetry['steering']:.3f}")
                print(f"  Throttle: {telemetry['throttle']:.2f}")
                print(f"  Brake: {telemetry['brake']:.2f}")
                print(f"  Gear: {telemetry['gear']}, RPM: {int(telemetry['rpm'])}")
            return True
        else:
            print("Failed to capture frame")
            return False

    def cleanup(self):
        """Cleanup resources"""
        if hasattr(self, 'asm'):
            self.asm.close()
        if self.camera:
            del self.camera

def main():
    print("Screenshot capture ready. Press 'c' to capture, 'q' to quit.")
    print("Make sure Assetto Corsa is running and in a session!")
    
    try:
        capture = ScreenshotCapture()
        
        # Test telemetry connection
        telemetry = capture.read_telemetry_data()
        if telemetry is None:
            print("WARNING: Cannot connect to Assetto Corsa telemetry!")
            print("Make sure Assetto Corsa is running and you're in a session.")
            print("Screenshots will still work but without telemetry data.")
        else:
            print(f"✓ Connected to AC telemetry. Current speed: {telemetry['speed']:.1f} km/h")
        
        while True:
            try:
                if keyboard.is_pressed('c'):
                    capture.capture_screenshot()
                    time.sleep(0.2)  # Prevent multiple captures
                    
                elif keyboard.is_pressed('q'):
                    print("Exiting...")
                    break
                    
                time.sleep(0.01)  # Small delay to reduce CPU usage
                
            except KeyboardInterrupt:
                print("\nExiting...")
                break
                
    except RuntimeError as e:
        print(f"Error: {e}")
        return
    except Exception as e:
        print(f"Unexpected error: {e}")
        return
    finally:
        if 'capture' in locals():
            capture.cleanup()

if __name__ == "__main__":
    main()


Screenshot capture ready. Press 'c' to capture, 'q' to quit.
Make sure Assetto Corsa is running and in a session!
DXCam and AC telemetry initialized successfully
✓ Connected to AC telemetry. Current speed: 58.9 km/h


Exception ignored in: <function _compointer_base.__del__ at 0x00000140DED72FC0>
Traceback (most recent call last):
  File "c:\Users\abdul\Desktop\ML\projects\SelfDriving\.venv\Lib\site-packages\comtypes\_post_coinit\unknwn.py", line 284, in __del__
    self.Release()  # type: ignore
  File "c:\Users\abdul\Desktop\ML\projects\SelfDriving\.venv\Lib\site-packages\comtypes\_post_coinit\unknwn.py", line 416, in Release
    return self.__com_Release()  # type: ignore
OSError: exception: access violation writing 0x00005B0000000000


: 